Import TensorFlow and the TensorBoard HParams plugin:

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel('WARNING')
from tensorboard.plugins.hparams import api as hp

In [ ]:
import itertools
from google.colab import widgets
import random
import numpy as np
import statistics 
import scipy.stats
from tensorflow.keras import Input, layers, Model, utils, initializers, losses, optimizers, Sequential, callbacks, backend
import pickle
import matplotlib.pyplot as plt
import tensorflow.keras as K
from sklearn.metrics import recall_score as recall
from sklearn.metrics import confusion_matrix as confusion

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/BTP - Dev Priya and Kushagra/Speech Emotion Recognition/'

/content/drive/My Drive/BTP - Dev Priya and Kushagra/Speech Emotion Recognition


In [ ]:
#Path to folder in which we have stored datasets for each session
dataset_dir = 'Datasets/TD'

In [ ]:
speaker_list = ['1F', '1M', '2F', '2M', '3F', '3M', '4F', '4M', '5F', '5M']

# **Util Functions**




In [ ]:
def retrieve_preprocessed_data(session):
    idx = session - 1
    speaker = speaker_list[idx]
    next_speaker = speaker_list[(idx+1)%10]
    file_code = 'Valid_' + speaker + 'Test_' + next_speaker
    data_file_name = dataset_dir + '/data' + file_code + '.pkl'
    print(data_file_name)

    f = open(data_file_name, 'rb')
    output = pickle.load(f)
    train_features, train_labels = output[0], output[1]  
    test_features, test_labels, test_segments_per_utterance = output[2], output[3], output[5]
    valid_features, valid_labels, valid_segments_per_utterance = output[6], output[7], output[9]
    train_td, test_td, valid_td = output[10], output[11], output[12]  
    f.close()

    train_features = tf.convert_to_tensor(train_features, dtype=tf.float32)
    valid_features = tf.convert_to_tensor(valid_features, dtype=tf.float32)
    test_features = tf.convert_to_tensor(test_features, dtype=tf.float32)
    train_td = tf.convert_to_tensor(train_td, dtype=tf.float32)
    test_td = tf.convert_to_tensor(test_td, dtype=tf.float32)
    valid_td = tf.convert_to_tensor(valid_td, dtype=tf.float32)
    
    train_labels = tf.one_hot(train_labels, 4, dtype=tf.float32)
    valid_labels = tf.one_hot(valid_labels, 4, dtype=tf.float32)
    test_labels = tf.one_hot(test_labels, 4, dtype=tf.float32)

    train_labels = tf.reshape(train_labels, [train_labels.shape[0], 4])
    valid_labels = tf.reshape(valid_labels, [valid_labels.shape[0], 4])
    test_labels = tf.reshape(test_labels, [test_labels.shape[0], 4])

    return train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance, train_td, test_td, valid_td

In [ ]:
def training(model, num_epochs=1200):
    i=0
    best_valid_accuracy = 0
    best_epoch = 0

    while i< num_epochs:
        start = (i*60)%train_features.shape[0]
        end = min(start+60, train_features.shape[0])
        train_batch = train_features[start:end,:,:,:]
        train_batch_label = train_labels[start:end,:]
        train_batch_td = train_td[start:end,:]
        loss = (model.train_on_batch([train_batch, train_batch_td, train_batch_label], train_batch_label, return_dict=True))

        if((i+1)%5==0):
            valid_acc_uw, valid_conf = evaluate(model, valid_features, valid_td, 
                                                valid_segments_per_utterance, 
                                                valid_labels)
            
            if valid_acc_uw > best_valid_accuracy:
                best_epoch = i+1
                best_valid_accuracy = valid_acc_uw
                test_accuracy, test_conf = evaluate(model, test_features, test_td, 
                                                    test_segments_per_utterance, 
                                                    test_labels)
            
        i += 1

    print('*'*30)
    print("Best Epoch: %05d" %(best_epoch))
    print("Best Valid Accuracy: " + str(best_valid_accuracy))
    print("Test_UA: " + str(test_accuracy))    
    print('Test Confusion Matrix:["ang","sad","hap","neu"]')
    print(test_conf)

    return test_accuracy

In [ ]:
def evaluate(model, spec_features, td_features, segments_per_utterance, labels):
    dummy = np.zeros((spec_features.shape[0], 4))
    y_pred = np.empty((len(segments_per_utterance),4),dtype=np.float32)
    y_pred_segments, c_loss = model.predict([spec_features, td_features, dummy])
    index=0
    for j in range(len(segments_per_utterance)):
        y_pred[j,:] = np.max(y_pred_segments[index:index+segments_per_utterance[j][0],:],0) 
        index+=(segments_per_utterance[j][0])

    acc_uw = recall(np.argmax(labels,1),np.argmax(y_pred,1),average='macro')
    conf = confusion(np.argmax(labels, 1),np.argmax(y_pred,1))
    return acc_uw, conf

## Define Model

In [ ]:
class CustomAttention(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        self.hidden_size = kwargs.pop('hidden_size')
        super(CustomAttention, self).__init__(*args, **kwargs)
        self.W_omega = tf.Variable(
            initializers.GlorotNormal(seed=0)(shape=[self.hidden_size, 1]), 
            dtype=tf.float32,
            trainable=True,
            name="W_omega")
        self.b_omega = tf.Variable(
            initializers.GlorotNormal(seed=0)(shape=[1]), 
            dtype=tf.float32,
            trainable=True,
            name="b_omega")
        self.u_omega = tf.Variable(
            initializers.GlorotNormal(seed=0)(shape=[1]), 
            dtype=tf.float32,
            trainable=True,
            name="u_omega")
    
    def call(self, inputs, **kwargs):
        v = tf.sigmoid(tf.tensordot(inputs, self.W_omega, axes=1) + self.b_omega)
        vu = tf.tensordot(v, self.u_omega, axes=1)
        alphas = layers.Softmax()(vu)
        return tf.reduce_sum(inputs * tf.expand_dims(alphas, -1), 1)
        
    def get_config(self):
        config = ({
            'hidden_size': self.hidden_size 
        })
        base_config = super(CustomAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
class CenterLossLayer(tf.keras.layers.Layer):

    def __init__(self, alpha=0.5, **kwargs):
        super().__init__(**kwargs)
        self.alpha = alpha

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers',
                                       shape=(4, 64),
                                       initializer='uniform',
                                       trainable=False)
        super().build(input_shape)

    def call(self, x, mask=None):
        delta_centers = K.backend.dot(K.backend.transpose(x[1]), (K.backend.dot(x[1], self.centers) - x[0]))  # 10x2
        center_counts = K.backend.sum(K.backend.transpose(x[1]), axis=1, keepdims=True) + 1  # 10x1
        delta_centers /= center_counts
        new_centers = self.centers - self.alpha * delta_centers
        self.add_update((self.centers, new_centers))

        self.result = x[0] - K.backend.dot(x[1], self.centers)
        self.result = K.backend.sum(self.result ** 2, axis=1, keepdims=True) #/ K.dot(x[1], center_counts)
        return self.result # Nx1

    def compute_output_shape(self, input_shape):
        return K.backend.int_shape(self.result)
      
    def get_config(self):
        base_config = super(CenterLossLayer, self).get_config()
        return dict(list(base_config.items()))

In [ ]:
def zero_loss(y_true, y_pred):
    return 0.5 * K.backend.sum(y_pred, axis=0)

In [ ]:
cols_to_keep = [
        'pcm_LOGenergy_sma_range', 'pcm_LOGenergy_sma_minameandist',
       'pcm_LOGenergy_sma_linregc2', 'pcm_LOGenergy_sma_linregerrQ',
       'pcm_LOGenergy_sma_variance', 'pcm_LOGenergy_sma_quartile1',
       'pcm_LOGenergy_sma_quartile2', 'pcm_LOGenergy_sma_quartile3',
       'pcm_LOGenergy_sma_percentile95.0', 'pcm_LOGenergy_sma_percentile98.0',
       'pcm_LOGenergy_sma_peakMean', 'pcm_zcr_sma_quartile1',
       'pcm_zcr_sma_quartile2', 'pcm_zcr_sma_iqr1-2',
       'voiceProb_sma_quartile2', 'pcm_LOGenergy_sma_de_linregc2',
       'pcm_LOGenergy_sma_de_quartile1', 'pcm_LOGenergy_sma_de_quartile2',
       'pcm_LOGenergy_sma_de_percentile95.0', 'pcm_LOGenergy_sma_de_zcr',
       'pcm_LOGenergy_sma_de_peakMeanMeanDist', 'pcm_LOGenergy_sma_de_amean',
       'pcm_LOGenergy_sma_de_absmean', 'pcm_zcr_sma_de_iqr1-3',
       'voiceProb_sma_de_quartile2', 'voiceProb_sma_de_meanPeakDist',
       'F0env_sma_de_skewness', 'pcm_LOGenergy_sma_de_de_qregc1',
       'pcm_LOGenergy_sma_de_de_kurtosis', 'pcm_LOGenergy_sma_de_de_zcr'
]

In [ ]:
input_shape_spec = (300, 40, 3)
input_shape_td = (300, len(cols_to_keep))

In [ ]:
def build_model(hparams):
    
    aux_input = Input(shape=(4,), name='labels')
    model_input_spec = Input(shape=input_shape_spec, name='log_mel_data')
    model_input_td = Input(shape=input_shape_td, name='td_data')

    x = layers.Conv2D(filters=hparams['FILTER_CONV1'], kernel_size=hparams['KERNEL_CONV1'], 
                      strides=hparams['STRIDE_CONV1'], bias_initializer=hparams['BIAS_INIT'], 
                      kernel_initializer=hparams['KERNEL_INIT'],
                      padding=hparams['PADDING_CONV1'], name='CONV1')(model_input_spec)
    if hparams['BN_CONV1']:
        x = layers.BatchNormalization(momentum=hparams['BN_MOMENTUM'], 
                                      name='BN_CONV1')(x)
    x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'],
                         name='LEAKY_CONV1')(x)
    if hparams['POOL_CONV1'] == 'max':
        x = layers.MaxPooling2D(pool_size=hparams['SIZE_POOL_CONV1'], 
                                strides=hparams['SIZE_POOL_CONV1'], 
                                padding=hparams['PADDING_POOL_CONV1'], 
                                name='POOL_CONV1')(x)
    else:
        x = layers.AveragePooling2D(pool_size=hparams['SIZE_POOL_CONV1'], 
                                    strides=hparams['SIZE_POOL_CONV1'], 
                                    padding=hparams['PADDING_POOL_CONV1'], 
                                    name='POOL_CONV1')(x)

    conv_layer_output = x

    for i in range(hparams['NUM_DIL_LAYERS']):
        x = layers.Conv2D(filters=hparams['FILTER_CONV_UFLB'], 
                          kernel_size=hparams['KERNEL_CONV_UFLB'], 
                          strides=hparams['STRIDE_CONV_UFLB'],
                          bias_initializer=hparams['BIAS_INIT'],
                          kernel_initializer=hparams['KERNEL_INIT'],
                          dilation_rate=hparams['DIL_RATE_CONV_UFLB'], 
                          padding=hparams['PADDING_CONV_UFLB'], 
                          name='CONV_UFLB_'+str(i+1))(x)
        x = layers.BatchNormalization(momentum=hparams['BN_MOMENTUM'], 
                                      name='BN_CONV_UFLB_'+str(i+1))(x)
        x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'],
                             name='LEAKY_CONV_UFLB_'+str(i+1))(x)

    skip_layer_output = layers.Conv2D(filters=hparams['FILTER_CONV_UFLB'], 
                                      kernel_size=hparams['KERNEL_CONV_UFLB'], 
                                      strides=hparams['STRIDE_CONV_UFLB'],
                                      bias_initializer=hparams['BIAS_INIT'],
                                      kernel_initializer=hparams['KERNEL_INIT'],
                                      dilation_rate=hparams['DIL_RATE_CONV_UFLB'], 
                                      padding=hparams['PADDING_CONV_UFLB'], 
                                      name='CONV_ALT')(conv_layer_output)
    skip_layer_output = layers.BatchNormalization(momentum=hparams['BN_MOMENTUM'], 
                                                  name='BN_CONV_ALT')(skip_layer_output)
    if hparams['LEAKY_CONV_ALT']:
        skip_layer_output = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'],
                            name='LEAKY_CONV_ALT')(skip_layer_output)
    x = layers.Add(name='skip_connection')([x, skip_layer_output])

    x = layers.Dropout(rate=hparams['DROP1_RATE'], name='DROPOUT1')(x)
    
    time_step = x.shape[1]
    linear_units = x.shape[2]*x.shape[3]
    x = tf.reshape(x,[-1,time_step,linear_units])

    if hparams['LINEAR_UNITS'] != 0:
        x = tf.reshape(x, [-1, linear_units])
        x = layers.Dense(hparams['LINEAR_UNITS'], activation = 'linear', 
                         name='LINEAR')(x)
        x = tf.reshape(x, [-1, time_step, hparams['LINEAR_UNITS']])
    
    if hparams['BN_LINEAR']:
        x = layers.BatchNormalization(momentum=hparams['BN_MOMENTUM'], 
                                      name='BN_LINEAR')(x)
    if hparams['LEAKY_LINEAR']:
        x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'],
                             name='LEAKY_LINEAR')(x)
    x = layers.Bidirectional(layers.LSTM(units=hparams['BILSTM_UNITS_SPEC'],
                                         bias_initializer=hparams['BIAS_INIT'],
                                         kernel_initializer=hparams['KERNEL_INIT'], 
                                         return_sequences=True))(x)
    x = CustomAttention(hidden_size=x.shape[2])(x)

    for i, n in enumerate(hparams['UNITS_FCN_SPEC']):
        x = layers.Dense(units=n, activation="linear", 
                         name='FCN_SPEC_DENSE'+str(i+1))(x)
        x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'], 
                             name='FCN_SPEC_LEAKY'+str(i+1))(x)

    x = layers.Dropout(rate=hparams['DROP2_RATE'], name='DROPOUT2')(x)

    model_spec = Model(inputs=model_input_spec, outputs=x, name="model_spec") 

    y = layers.Bidirectional(layers.LSTM(hparams['BILSTM_UNITS_TD'],
                                         bias_initializer=hparams['BIAS_INIT'],
                                         kernel_initializer=hparams['KERNEL_INIT'],  
                                         return_sequences=True))(model_input_td)
    y = CustomAttention(hidden_size=y.shape[2])(y)
    for i, n in enumerate(hparams['UNITS_FCN_TD']):
        x = layers.Dense(units=n, activation="linear", 
                         name='FCN_TD_DENSE'+str(i+1))(x)
        x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'], 
                             name='FCN_TD_LEAKY'+str(i+1))(x)
    
    x = layers.Dropout(rate=hparams['DROP3_RATE'], name='DROPOUT3')(x)

    model_td = Model(inputs=model_input_td, outputs=y, name="model_td")

    x = tf.concat([model_td.output, model_spec.output], axis=1)
    for i, n in enumerate(hparams['UNITS_FCN_CONCAT']):
        x = layers.Dense(units=n, activation="linear", 
                         name='FCN_CONCAT_DENSE'+str(i+1))(x)
        x = layers.LeakyReLU(alpha=hparams['LEAKY_ALPHA'], 
                             name='FCN_CONCAT_LEAKY'+str(i+1))(x)
    output = layers.Dense(4, activation="linear")(x)
    center_loss = CenterLossLayer(alpha=hparams['CL_APLHA'], 
                                  name='center_loss_layer')([x, aux_input])
    
    model = Model(inputs=[model_spec.input, model_td.input, aux_input], 
                  outputs=[output, center_loss], name="adrnn_with_td_model") 
    model.compile(
        loss=[losses.CategoricalCrossentropy(from_logits=True), zero_loss],
        loss_weights = [1, hparams['CL_WEIGHT']],
        optimizer=optimizers.Adam(
            learning_rate=hparams['LR']
        ),
        metrics=['categorical_accuracy'],
    )
    # print(model.summary())

    return model


# Experiment setup 

## Define hyperparams

In [ ]:
params_dict_of_lists = {
'FILTER_CONV1' : [128],
'KERNEL_CONV1' : [(3, 3)],
'STRIDE_CONV1' : [(1, 1)],
'BIAS_INIT' : ['ones'],
'KERNEL_INIT' : ['glorot_normal'],
'PADDING_CONV1' : ['SAME'],
'BN_CONV1' : [True],
'BN_MOMENTUM' : [0.9],
'LEAKY_ALPHA' : [0.01],
'POOL_CONV1' : ['max', 'avg'],
'SIZE_POOL_CONV1' : [(2, 4)],
'PADDING_POOL_CONV1' : ['VALID'],
'NUM_DIL_LAYERS' : range(3, 6, 1),
'FILTER_CONV_UFLB' : [128],
'KERNEL_CONV_UFLB' :  [(3, 3)],
'STRIDE_CONV_UFLB' : [(1, 1)],
'DIL_RATE_CONV_UFLB' : [(2, 2)],
'PADDING_CONV_UFLB' : ['SAME'],
'LEAKY_CONV_ALT' : [False],
'DROP1_RATE' : [0.0],
'LINEAR_UNITS' : [0],
'BN_LINEAR' : [False],
'LEAKY_LINEAR' : [False],
'BILSTM_UNITS_SPEC' : [256],
'UNITS_FCN_SPEC' : [[512, 128, 64], [512, 64]],
'DROP2_RATE' : [0.0],
'BILSTM_UNITS_TD' : [32, 64],
'UNITS_FCN_TD' : [[64]],
'DROP3_RATE' : [0.0],
'UNITS_FCN_CONCAT' : [[64]],
'CL_APLHA' : [0.5],
'CL_WEIGHT' : [0.01, 0.1],
'LR' : [0.00001],
}

In [ ]:
keys, values = zip(*params_dict_of_lists.items())
params_list_of_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]
print(len(params_list_of_dicts))
print(params_list_of_dicts[0])

48
{'FILTER_CONV1': 128, 'KERNEL_CONV1': (3, 3), 'STRIDE_CONV1': (1, 1), 'BIAS_INIT': 'ones', 'KERNEL_INIT': 'glorot_normal', 'PADDING_CONV1': 'SAME', 'BN_CONV1': True, 'BN_MOMENTUM': 0.9, 'LEAKY_ALPHA': 0.01, 'POOL_CONV1': 'max', 'SIZE_POOL_CONV1': (2, 4), 'PADDING_POOL_CONV1': 'VALID', 'NUM_DIL_LAYERS': 3, 'FILTER_CONV_UFLB': 128, 'KERNEL_CONV_UFLB': (3, 3), 'STRIDE_CONV_UFLB': (1, 1), 'DIL_RATE_CONV_UFLB': (2, 2), 'PADDING_CONV_UFLB': 'SAME', 'LEAKY_CONV_ALT': False, 'DROP1_RATE': 0.0, 'LINEAR_UNITS': 0, 'BN_LINEAR': False, 'LEAKY_LINEAR': False, 'BILSTM_UNITS_SPEC': 256, 'UNITS_FCN_SPEC': [512, 128, 64], 'DROP2_RATE': 0.0, 'BILSTM_UNITS_TD': 32, 'UNITS_FCN_TD': [64], 'DROP3_RATE': 0.0, 'UNITS_FCN_CONCAT': [64], 'CL_APLHA': 0.5, 'CL_WEIGHT': 0.01, 'LR': 1e-05}


## Acquire data 

In [ ]:
session = 9
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance, train_td, test_td, valid_td = retrieve_preprocessed_data(session)

Datasets/TD/dataValid_5FTest_5M.pkl


## Experiment 0-9

In [ ]:
# experiment_num = 0
# best_accuracy = 0
# best_experiment = 0
# for hparams in params_list_of_dicts[:10]:
#     print('--- Starting trial: %s' % experiment_num)
#     for run in range(5):
#         model = build_model(hparams)
#         accuracy = training(model, 500)
#         if best_accuracy < accuracy:
#             best_accuracy = accuracy
#             best_experiment = experiment_num
#     experiment_num += 1
#     if(experiment_num%5 == 0):
#         print('-------------- RESULTS TILL NOW: ------------')
#         print('best_experiment: %d' % best_experiment)
#         print('best_accuracy' + str(best_accuracy))

--- Starting trial: 0
******************************
Best Epoch: 00480
Training accuracy: 1.3070027828216553
Test_UA: 0.5229392911010559
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  4  1  0]
 [ 8 43  9  0]
 [15  2 31  2]
 [18 56 43 19]]
******************************
Best Epoch: 00485
Training accuracy: 1.33749520778656
Test_UA: 0.468644419306184
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 6  2  0  5]
 [ 5 44  1 10]
 [15  0  2 33]
 [14 35  0 87]]
******************************
Best Epoch: 00475
Training accuracy: 1.3029179573059082
Test_UA: 0.4489536199095022
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  4  0  2]
 [ 8 45  0  7]
 [22  0  0 28]
 [20 47  0 69]]
******************************
Best Epoch: 00255
Training accuracy: 1.4498814344406128
Test_UA: 0.4898793363499246
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 5  2  6  0]
 [ 1 40 19  0]
 [ 4  0 41  5]
 [ 7 34 83 12]]
******************************
Best Epoch: 00465
Training accuracy: 1.335711359977722

## Experiment 10-19

In [ ]:
# trial 10

--- Starting trial: 10
******************************
Best Epoch: 00485
Training accuracy: 1.3113247156143188
Test_UA: 0.5064102564102564
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  1  0  3]
 [11 41  5  3]
 [16  1 20 13]
 [21 56 25 34]]
******************************
Best Epoch: 00500
Training accuracy: 1.3289967775344849
Test_UA: 0.49468325791855206
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 5  1  2  5]
 [ 2 39  6 13]
 [ 4  0 20 26]
 [ 7 36 19 74]]
******************************
Best Epoch: 00480
Training accuracy: 1.337017297744751
Test_UA: 0.5282767722473605
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 6  4  3  0]
 [ 2 44  9  5]
 [ 7  0 29 14]
 [15 55 20 46]]
******************************
Best Epoch: 00500
Training accuracy: 1.315685749053955
Test_UA: 0.5540478883861236
Test Confusion Matrix:["ang","sad","hap","neu"]
[[10  2  0  1]
 [14 40  2  4]
 [14  0 21 15]
 [21 45 21 49]]
******************************
Best Epoch: 00370
Training accuracy: 1.317462801933

In [ ]:
# trial 11 to 18

--- Starting trial: 11
******************************
Best Epoch: 00495
Training accuracy: 1.527214527130127
Test_UA: 0.39510558069381596
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  4  1  1]
 [ 8 40  1 11]
 [22  3  7 18]
 [30 62 12 32]]
******************************
Best Epoch: 00465
Training accuracy: 1.6371352672576904
Test_UA: 0.3417986425339367
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 12   0   0   1]
 [ 31  24   0   5]
 [ 49   0   0   1]
 [113  17   0   6]]
******************************
Best Epoch: 00475
Training accuracy: 1.6129590272903442
Test_UA: 0.36378959276018097
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 1  2  3  7]
 [ 0 33  4 23]
 [ 1  2 12 35]
 [12 31 13 80]]
******************************
Best Epoch: 00385
Training accuracy: 1.6076619625091553
Test_UA: 0.3881447963800905
Test Confusion Matrix:["ang","sad","hap","neu"]
[[  7   0   0   6]
 [  7   0   3  50]
 [  9   2  11  28]
 [ 20   3   5 108]]
******************************
Best Epoch: 00335

KeyboardInterrupt: ignored

## Experiment 20-29

In [ ]:
# trial 20

--- Starting trial: 20
******************************
Best Epoch: 00455
Training accuracy: 1.3219670057296753
Test_UA: 0.536615761689291
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  5  0  0]
 [ 6 46  3  5]
 [10  0 18 22]
 [14 51 16 55]]
******************************
Best Epoch: 00485
Training accuracy: 1.305938959121704
Test_UA: 0.5346700603318251
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  3  0  1]
 [ 7 43  6  4]
 [14  0 17 19]
 [21 47 15 53]]
******************************
Best Epoch: 00415
Training accuracy: 1.283571481704712
Test_UA: 0.5315818250377073
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  4  0  0]
 [12 43  4  1]
 [17  3 23  7]
 [23 58 20 35]]
******************************
Best Epoch: 00455
Training accuracy: 1.3315467834472656
Test_UA: 0.4631843891402715
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  5  0  0]
 [ 8 48  2  2]
 [23  7  9 11]
 [24 68  9 35]]
******************************
Best Epoch: 00445
Training accuracy: 1.32345306873321

In [ ]:
# trial 22

--- Starting trial: 22
******************************
Best Epoch: 00495
Training accuracy: 1.3090206384658813
Test_UA: 0.4825320512820513
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  2  0  4]
 [ 4 40  2 14]
 [13  1  5 31]
 [15 31  5 85]]
******************************
Best Epoch: 00475
Training accuracy: 1.2973321676254272
Test_UA: 0.4835614630467572
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  3  1  2]
 [ 5 43  5  7]
 [15  1 13 21]
 [18 48 13 57]]
******************************
Best Epoch: 00495
Training accuracy: 1.3113842010498047
Test_UA: 0.5145324283559578
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  1  2]
 [10 38  3  9]
 [16  0 14 20]
 [22 30 12 72]]
******************************
Best Epoch: 00385
Training accuracy: 1.333693265914917
Test_UA: 0.5138216440422323
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  4  0  1]
 [ 7 43  2  8]
 [17  1 13 19]
 [18 46  9 63]]
******************************
Best Epoch: 00300
Training accuracy: 1.359823703765

In [ ]:
# for experiment_num in range(24, 30):
#     if experiment_num %2 !=0:
#         continue
#     hparams = params_list_of_dicts[experiment_num]
#     print('--- Starting trial: %s' % experiment_num)
#     for run in range(5):
#         model = build_model(hparams)
#         accuracy = training(model, 500)
#         if best_accuracy < accuracy:
#             best_accuracy = accuracy
#             best_experiment = experiment_num
#     if((experiment_num+1)%5 == 0):
#         print('-------------- RESULTS TILL NOW: ------------')
#         print('best_experiment: %d' % best_experiment)
#         print('best_accuracy' + str(best_accuracy))

--- Starting trial: 24
******************************
Best Epoch: 00495
Training accuracy: 1.3327853679656982
Test_UA: 0.45709087481146304
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  4  1  1]
 [ 6 49  5  0]
 [20  7 13 10]
 [23 70 14 29]]
******************************
Best Epoch: 00460
Training accuracy: 1.341917872428894
Test_UA: 0.48252262443438915
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  3  0  2]
 [ 7 45  5  3]
 [21  3 15 11]
 [31 55 14 36]]
******************************
Best Epoch: 00390
Training accuracy: 1.3889275789260864
Test_UA: 0.4949528657616893
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 5  2  1  5]
 [ 0 43  3 14]
 [ 3  0  9 38]
 [ 5 30  6 95]]
******************************
Best Epoch: 00485
Training accuracy: 1.2878586053848267
Test_UA: 0.5508861236802414
Test Confusion Matrix:["ang","sad","hap","neu"]
[[10  2  1  0]
 [12 40  6  2]
 [15  0 20 15]
 [26 41 19 50]]
******************************
Best Epoch: 00475
Training accuracy: 1.3245941400

## Experiment 30-39

In [ ]:
params_list_of_dicts[26]

{'BIAS_INIT': 'ones',
 'BILSTM_UNITS_SPEC': 256,
 'BILSTM_UNITS_TD': 64,
 'BN_CONV1': True,
 'BN_LINEAR': False,
 'BN_MOMENTUM': 0.9,
 'CL_APLHA': 0.5,
 'CL_WEIGHT': 0.01,
 'DIL_RATE_CONV_UFLB': (2, 2),
 'DROP1_RATE': 0.0,
 'DROP2_RATE': 0.0,
 'DROP3_RATE': 0.0,
 'FILTER_CONV1': 128,
 'FILTER_CONV_UFLB': 128,
 'KERNEL_CONV1': (3, 3),
 'KERNEL_CONV_UFLB': (3, 3),
 'KERNEL_INIT': 'glorot_normal',
 'LEAKY_ALPHA': 0.01,
 'LEAKY_CONV_ALT': False,
 'LEAKY_LINEAR': False,
 'LINEAR_UNITS': 0,
 'LR': 1e-05,
 'NUM_DIL_LAYERS': 3,
 'PADDING_CONV1': 'SAME',
 'PADDING_CONV_UFLB': 'SAME',
 'PADDING_POOL_CONV1': 'VALID',
 'POOL_CONV1': 'avg',
 'SIZE_POOL_CONV1': (2, 4),
 'STRIDE_CONV1': (1, 1),
 'STRIDE_CONV_UFLB': (1, 1),
 'UNITS_FCN_CONCAT': [64],
 'UNITS_FCN_SPEC': [512, 128, 64],
 'UNITS_FCN_TD': [64]}

In [ ]:
# for experiment_num in range(30, 40):
#     if experiment_num %2 !=0:
#         continue
#     hparams = params_list_of_dicts[experiment_num]
#     print('--- Starting trial: %s' % experiment_num)
#     for run in range(5):
#         model = build_model(hparams)
#         accuracy = training(model, 500)
#         if best_accuracy < accuracy:
#             best_accuracy = accuracy
#             best_experiment = experiment_num
#     if((experiment_num+1)%5 == 0):
#         print('-------------- RESULTS TILL NOW: ------------')
#         print('best_experiment: %d' % best_experiment)
#         print('best_accuracy' + str(best_accuracy))

--- Starting trial: 30
******************************
Best Epoch: 00490
Training accuracy: 1.3210523128509521
Test_UA: 0.4842835595776772
Test Confusion Matrix:["ang","sad","hap","neu"]
[[11  1  0  1]
 [13 41  0  6]
 [34  1  2 13]
 [42 44  0 50]]
******************************
Best Epoch: 00475
Training accuracy: 1.2923535108566284
Test_UA: 0.5444588989441931
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  3  0  2]
 [ 4 41  9  6]
 [12  0 23 15]
 [21 41 17 57]]
******************************
Best Epoch: 00460
Training accuracy: 1.27616548538208
Test_UA: 0.514854826546003
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 5  2  2  4]
 [ 0 41  7 12]
 [ 6  0 22 22]
 [ 9 36 16 75]]
******************************
Best Epoch: 00420
Training accuracy: 1.3123446702957153
Test_UA: 0.4769532428355957
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 6  2  0  5]
 [ 4 41  4 11]
 [10  0  8 32]
 [14 34  6 82]]
******************************
Best Epoch: 00475
Training accuracy: 1.32876145839691

In [ ]:
best_experiment = 26
best_accuracy = 0.5652130467571644

In [ ]:
for experiment_num in range(38, 40):
    if experiment_num %2 !=0:
        continue
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 38
******************************
Best Epoch: 00460
Training accuracy: 1.3195058107376099
Test_UA: 0.4686896681749623
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [12 41  0  7]
 [28  1  4 17]
 [38 41  0 57]]
******************************
Best Epoch: 00500
Training accuracy: 1.316596508026123
Test_UA: 0.5325377073906485
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  3  0  1]
 [ 9 43  2  6]
 [16  0 14 20]
 [22 43 11 60]]
******************************
Best Epoch: 00500
Training accuracy: 1.2982851266860962
Test_UA: 0.553380467571644
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  2  1]
 [ 7 40  8  5]
 [13  0 19 18]
 [19 29 13 75]]
******************************
Best Epoch: 00500
Training accuracy: 1.296025037765503
Test_UA: 0.5269438159879336
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  2  1  3]
 [ 2 43  4 11]
 [ 7  0 18 25]
 [11 44 14 67]]
******************************
Best Epoch: 00285
Training accuracy: 1.44073307514190

## Experiment 40-47

In [ ]:
for experiment_num in range(40, 48):
    if experiment_num %2 !=0:
        continue
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 40
******************************
Best Epoch: 00485
Training accuracy: 1.317620038986206
Test_UA: 0.5305957767722473
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  4  0  2]
 [ 4 44  4  8]
 [13  0 19 18]
 [18 40 14 64]]
******************************
Best Epoch: 00470
Training accuracy: 1.3179906606674194
Test_UA: 0.5348265460030166
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  4  0  1]
 [ 6 44  3  7]
 [12  2 16 20]
 [18 40 14 64]]
******************************
Best Epoch: 00450
Training accuracy: 1.2919063568115234
Test_UA: 0.5245079185520363
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  1  2]
 [ 5 39  9  7]
 [12  0 17 21]
 [20 38 11 67]]
******************************
Best Epoch: 00490
Training accuracy: 1.3195112943649292
Test_UA: 0.5234445701357466
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [ 8 39  3 10]
 [17  0 10 23]
 [23 28 10 75]]
******************************
Best Epoch: 00490
Training accuracy: 1.358905076980

In [ ]:
best_experiment = 26
best_accuracy = 0.5652130467571644

In [ ]:
for experiment_num in range(40, 48):
    if experiment_num %2 !=0:
        continue
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 40
******************************
Best Epoch: 00440
Training accuracy: 1.3329951763153076
Test_UA: 0.5821153846153846
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  2  2  2]
 [ 5 39  7  9]
 [ 5  0 32 13]
 [13 29 26 68]]
******************************
Best Epoch: 00485
Training accuracy: 1.528895616531372
Test_UA: 0.4297039969834088
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  4  0  1]
 [11 49  0  0]
 [31  6  0 13]
 [36 61  0 39]]
******************************
Best Epoch: 00370
Training accuracy: 1.5544072389602661
Test_UA: 0.44968891402714933
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  2  0  4]
 [ 7 39  0 14]
 [19  0  0 31]
 [23 30  0 83]]
******************************
Best Epoch: 00235
Training accuracy: 1.647612452507019
Test_UA: 0.543380467571644
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  0  3  2]
 [ 2 31 20  7]
 [ 6  0 37  7]
 [ 9 20 66 41]]
******************************
Best Epoch: 00300
Training accuracy: 1.7270278930664

KeyboardInterrupt: ignored

In [ ]:
for experiment_num in range(42, 48):
    if experiment_num %2 !=0:
        continue
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 42
******************************
Best Epoch: 00455
Training accuracy: 1.3176668882369995
Test_UA: 0.5075226244343891
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  0  3]
 [ 7 42  2  9]
 [17  0 10 23]
 [20 39  7 70]]
******************************
Best Epoch: 00490
Training accuracy: 1.3227612972259521
Test_UA: 0.5178355957767722
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 6  3  1  3]
 [ 1 47  4  8]
 [ 7  0 20 23]
 [11 53 14 58]]
******************************
Best Epoch: 00405
Training accuracy: 1.3589609861373901
Test_UA: 0.5127582956259427
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  4  0  0]
 [ 6 43 11  0]
 [17  2 31  0]
 [25 57 51  3]]
******************************
Best Epoch: 00415
Training accuracy: 1.304728627204895
Test_UA: 0.5596493212669683
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 6  3  2  2]
 [ 3 42 12  3]
 [ 6  0 34 10]
 [13 41 28 54]]
******************************
Best Epoch: 00485
Training accuracy: 1.296980619430

In [ ]:
params_list_of_dicts[42]

{'BIAS_INIT': 'ones',
 'BILSTM_UNITS_SPEC': 256,
 'BILSTM_UNITS_TD': 64,
 'BN_CONV1': True,
 'BN_LINEAR': False,
 'BN_MOMENTUM': 0.9,
 'CL_APLHA': 0.5,
 'CL_WEIGHT': 0.01,
 'DIL_RATE_CONV_UFLB': (2, 2),
 'DROP1_RATE': 0.0,
 'DROP2_RATE': 0.0,
 'DROP3_RATE': 0.0,
 'FILTER_CONV1': 128,
 'FILTER_CONV_UFLB': 128,
 'KERNEL_CONV1': (3, 3),
 'KERNEL_CONV_UFLB': (3, 3),
 'KERNEL_INIT': 'glorot_normal',
 'LEAKY_ALPHA': 0.01,
 'LEAKY_CONV_ALT': False,
 'LEAKY_LINEAR': False,
 'LINEAR_UNITS': 0,
 'LR': 1e-05,
 'NUM_DIL_LAYERS': 5,
 'PADDING_CONV1': 'SAME',
 'PADDING_CONV_UFLB': 'SAME',
 'PADDING_POOL_CONV1': 'VALID',
 'POOL_CONV1': 'avg',
 'SIZE_POOL_CONV1': (2, 4),
 'STRIDE_CONV1': (1, 1),
 'STRIDE_CONV_UFLB': (1, 1),
 'UNITS_FCN_CONCAT': [64],
 'UNITS_FCN_SPEC': [512, 128, 64],
 'UNITS_FCN_TD': [64]}

# Experiment setup: round 2

## Define hyperparams

In [ ]:
params_dict_of_lists = {
'FILTER_CONV1' : [128, 256],
'KERNEL_CONV1' : [(3, 3)],
'STRIDE_CONV1' : [(1, 1)],
'BIAS_INIT' : ['ones'],
'KERNEL_INIT' : ['glorot_normal'],
'PADDING_CONV1' : ['SAME'],
'BN_CONV1' : [True],
'BN_MOMENTUM' : [0.9, 0.5],
'LEAKY_ALPHA' : [0.01],
'POOL_CONV1' : ['max'],
'SIZE_POOL_CONV1' : [(2, 4)],
'PADDING_POOL_CONV1' : ['VALID'],
'NUM_DIL_LAYERS' : range(4, 6, 1),
'FILTER_CONV_UFLB' : [128],
'KERNEL_CONV_UFLB' :  [(3, 3)],
'STRIDE_CONV_UFLB' : [(1, 1)],
'DIL_RATE_CONV_UFLB' : [(2, 2)],
'PADDING_CONV_UFLB' : ['SAME'],
'LEAKY_CONV_ALT' : [False],
'DROP1_RATE' : [0.0],
'LINEAR_UNITS' : [0],
'BN_LINEAR' : [False],
'LEAKY_LINEAR' : [False],
'BILSTM_UNITS_SPEC' : [256],
'UNITS_FCN_SPEC' : [[512, 64]],
'DROP2_RATE' : [0.0],
'BILSTM_UNITS_TD' : [64],
'UNITS_FCN_TD' : [[64]],
'DROP3_RATE' : [0.0],
'UNITS_FCN_CONCAT' : [[64]],
'CL_APLHA' : [0.5],
'CL_WEIGHT' : [0.01],
'LR' : [0.00001],
}

In [ ]:
keys, values = zip(*params_dict_of_lists.items())
params_list_of_dicts_2 = [dict(zip(keys, v)) for v in itertools.product(*values)]
print(len(params_list_of_dicts_2))
print(params_list_of_dicts_2[0])

8
{'FILTER_CONV1': 128, 'KERNEL_CONV1': (3, 3), 'STRIDE_CONV1': (1, 1), 'BIAS_INIT': 'ones', 'KERNEL_INIT': 'glorot_normal', 'PADDING_CONV1': 'SAME', 'BN_CONV1': True, 'BN_MOMENTUM': 0.9, 'LEAKY_ALPHA': 0.01, 'POOL_CONV1': 'max', 'SIZE_POOL_CONV1': (2, 4), 'PADDING_POOL_CONV1': 'VALID', 'NUM_DIL_LAYERS': 4, 'FILTER_CONV_UFLB': 128, 'KERNEL_CONV_UFLB': (3, 3), 'STRIDE_CONV_UFLB': (1, 1), 'DIL_RATE_CONV_UFLB': (2, 2), 'PADDING_CONV_UFLB': 'SAME', 'LEAKY_CONV_ALT': False, 'DROP1_RATE': 0.0, 'LINEAR_UNITS': 0, 'BN_LINEAR': False, 'LEAKY_LINEAR': False, 'BILSTM_UNITS_SPEC': 256, 'UNITS_FCN_SPEC': [512, 64], 'DROP2_RATE': 0.0, 'BILSTM_UNITS_TD': 64, 'UNITS_FCN_TD': [64], 'DROP3_RATE': 0.0, 'UNITS_FCN_CONCAT': [64], 'CL_APLHA': 0.5, 'CL_WEIGHT': 0.01, 'LR': 1e-05}


In [ ]:
res = []
for d in params_list_of_dicts_2:
    if d not in params_list_of_dicts:
        res.append(d)
print(len(res))
params_list_of_dicts_2 = res

6


In [ ]:
params_list_of_dicts = res

## Acquire data 

In [ ]:
session = 9
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance, train_td, test_td, valid_td = retrieve_preprocessed_data(session)

Datasets/TD/dataValid_5FTest_5M.pkl


## Experiment 0-6

In [ ]:
best_experiment = 0
best_accuracy = 0

In [ ]:
for experiment_num in range(6):
    if experiment_num %2 !=0:
        continue
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 0
******************************
Best Epoch: 00480
Training accuracy: 1.295154333114624
Test_UA: 0.5348171191553545
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [12 40  3  5]
 [16  1 21 12]
 [29 40 18 49]]
******************************
Best Epoch: 00300
Training accuracy: 1.34696626663208
Test_UA: 0.5428110859728507
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 6  5  2  0]
 [ 5 48  3  4]
 [ 8  5 26 11]
 [11 55 17 53]]
******************************
Best Epoch: 00395
Training accuracy: 1.2937979698181152
Test_UA: 0.5105373303167421
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  4  1  0]
 [10 42  7  1]
 [16  2 22 10]
 [19 55 23 39]]
******************************
Best Epoch: 00490
Training accuracy: 1.2840791940689087
Test_UA: 0.5555920060331825
Test Confusion Matrix:["ang","sad","hap","neu"]
[[10  3  0  0]
 [11 40  4  5]
 [18  0 18 14]
 [21 40 17 58]]
******************************
Best Epoch: 00495
Training accuracy: 1.384702086448669

In [ ]:
for experiment_num in range(6):
    if experiment_num %2 ==0:
        continue
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 1
******************************
Best Epoch: 00200
Training accuracy: 1.3778035640716553
Test_UA: 0.5509351432880845
Test Confusion Matrix:["ang","sad","hap","neu"]
[[10  2  0  1]
 [ 7 38  9  6]
 [11  0 18 21]
 [21 29 26 60]]
******************************
Best Epoch: 00490
Training accuracy: 1.3266414403915405
Test_UA: 0.48558634992458527
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  0  3]
 [ 6 43  0 11]
 [15  1  0 34]
 [17 36  0 83]]
******************************
Best Epoch: 00335
Training accuracy: 1.3414677381515503
Test_UA: 0.5092816742081449
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 6  5  2  0]
 [ 3 45 10  2]
 [10  1 24 15]
 [21 49 19 47]]
******************************
Best Epoch: 00475
Training accuracy: 1.279187798500061
Test_UA: 0.4986651583710407
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [13 42  1  4]
 [21  3 11 15]
 [32 45  7 52]]
******************************
Best Epoch: 00495
Training accuracy: 1.345034480094

In [ ]:
for experiment_num in range(6):
    if experiment_num %2 !=0:
        continue
    hparams = params_list_of_dicts[experiment_num]
    hparams['POOL_CONV1'] = 'avg'
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 0
******************************
Best Epoch: 00440
Training accuracy: 1.3215985298156738
Test_UA: 0.5209690799396681
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [ 8 44  7  1]
 [18  1 16 15]
 [22 50 18 46]]
******************************
Best Epoch: 00500
Training accuracy: 1.267082929611206
Test_UA: 0.5301602564102563
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [ 7 44  3  6]
 [17  2 16 15]
 [29 47  9 51]]
******************************
Best Epoch: 00440
Training accuracy: 1.3067824840545654
Test_UA: 0.5066892911010559
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  3  0  2]
 [ 7 43  3  7]
 [20  2  9 19]
 [25 39  2 70]]
******************************
Best Epoch: 00460
Training accuracy: 1.3147499561309814
Test_UA: 0.5201112368024132
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [11 40  6  3]
 [16  0 21 13]
 [26 51 18 41]]
******************************
Best Epoch: 00400
Training accuracy: 1.3020877838134

In [ ]:
for experiment_num in range(6):
    if experiment_num %2 ==0:
        continue
    hparams = params_list_of_dicts[experiment_num]
    hparams['POOL_CONV1'] = 'avg'
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 1
******************************
Best Epoch: 00495
Training accuracy: 1.3190611600875854
Test_UA: 0.531184012066365
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  4  2  0]
 [ 3 44 10  3]
 [ 9  0 25 16]
 [15 53 20 48]]
******************************
Best Epoch: 00500
Training accuracy: 1.3224080801010132
Test_UA: 0.47275829562594274
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [12 37  4  7]
 [23  0  3 24]
 [31 34  0 71]]
******************************
Best Epoch: 00495
Training accuracy: 1.2937569618225098
Test_UA: 0.5136161387631976
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  4  0  0]
 [12 44  2  2]
 [20  1 16 13]
 [29 53 12 42]]
******************************
Best Epoch: 00460
Training accuracy: 1.3412518501281738
Test_UA: 0.5247869532428355
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  2  3  1]
 [ 2 44 14  0]
 [ 5  4 41  0]
 [ 9 61 65  1]]
******************************
Best Epoch: 00500
Training accuracy: 1.284985899925

# Experiment setup: round 2

## Define hyperparams

In [ ]:
params_dict_of_lists = {
'FILTER_CONV1' : [128],
'KERNEL_CONV1' : [(3, 3)],
'STRIDE_CONV1' : [(1, 1)],
'BIAS_INIT' : ['ones'],
'KERNEL_INIT' : ['glorot_normal'],
'PADDING_CONV1' : ['SAME'],
'BN_CONV1' : [True],
'BN_MOMENTUM' : [0.9],
'LEAKY_ALPHA' : [0.01, 0.3],
'POOL_CONV1' : ['max'],
'SIZE_POOL_CONV1' : [(2, 4)],
'PADDING_POOL_CONV1' : ['VALID'],
'NUM_DIL_LAYERS' : range(4, 6, 1),
'FILTER_CONV_UFLB' : [128],
'KERNEL_CONV_UFLB' :  [(3, 3), (5, 5)],
'STRIDE_CONV_UFLB' : [(1, 1)],
'DIL_RATE_CONV_UFLB' : [(2, 2)],
'PADDING_CONV_UFLB' : ['SAME'],
'LEAKY_CONV_ALT' : [False],
'DROP1_RATE' : [0.0, 0.25],
'LINEAR_UNITS' : [0],
'BN_LINEAR' : [False],
'LEAKY_LINEAR' : [False],
'BILSTM_UNITS_SPEC' : [256, 512],
'UNITS_FCN_SPEC' : [[512, 64], [1024, 512, 64]],
'DROP2_RATE' : [0.0],
'BILSTM_UNITS_TD' : [64],
'UNITS_FCN_TD' : [[64]],
'DROP3_RATE' : [0.0],
'UNITS_FCN_CONCAT' : [[64]],
'CL_APLHA' : [0.5],
'CL_WEIGHT' : [0.01],
'LR' : [0.00001],
}

In [ ]:
keys, values = zip(*params_dict_of_lists.items())
params_list_of_dicts_3 = [dict(zip(keys, v)) for v in itertools.product(*values)]
print(len(params_list_of_dicts_3))
print(params_list_of_dicts_3[0])

64
{'FILTER_CONV1': 128, 'KERNEL_CONV1': (3, 3), 'STRIDE_CONV1': (1, 1), 'BIAS_INIT': 'ones', 'KERNEL_INIT': 'glorot_normal', 'PADDING_CONV1': 'SAME', 'BN_CONV1': True, 'BN_MOMENTUM': 0.9, 'LEAKY_ALPHA': 0.01, 'POOL_CONV1': 'max', 'SIZE_POOL_CONV1': (2, 4), 'PADDING_POOL_CONV1': 'VALID', 'NUM_DIL_LAYERS': 4, 'FILTER_CONV_UFLB': 128, 'KERNEL_CONV_UFLB': (3, 3), 'STRIDE_CONV_UFLB': (1, 1), 'DIL_RATE_CONV_UFLB': (2, 2), 'PADDING_CONV_UFLB': 'SAME', 'LEAKY_CONV_ALT': False, 'DROP1_RATE': 0.0, 'LINEAR_UNITS': 0, 'BN_LINEAR': False, 'LEAKY_LINEAR': False, 'BILSTM_UNITS_SPEC': 256, 'UNITS_FCN_SPEC': [512, 64], 'DROP2_RATE': 0.0, 'BILSTM_UNITS_TD': 64, 'UNITS_FCN_TD': [64], 'DROP3_RATE': 0.0, 'UNITS_FCN_CONCAT': [64], 'CL_APLHA': 0.5, 'CL_WEIGHT': 0.01, 'LR': 1e-05}


In [ ]:
res = []
params_list_of_dicts = params_list_of_dicts + params_list_of_dicts_2
print(len(params_list_of_dicts))
for d in params_list_of_dicts_3:
    if d not in params_list_of_dicts:
        res.append(d)
print(len(res))
params_list_of_dicts_3 = res

54
62


In [ ]:
params_list_of_dicts = res

## Acquire data 

In [ ]:
session = 9
train_features, train_labels, valid_features, valid_labels, valid_segments_per_utterance, \
    test_features, test_labels, test_segments_per_utterance, train_td, test_td, valid_td = retrieve_preprocessed_data(session)

Datasets/TD/dataValid_5FTest_5M.pkl


## Experiment 0-6

In [ ]:
best_experiment = 0
best_accuracy = 0

In [ ]:
for experiment_num in range(20):
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 0
******************************
Best Epoch: 00285
Training accuracy: 1.358855962753296
Test_UA: 0.5489536199095022
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  2  2  2]
 [ 2 39 12  7]
 [ 7  0 25 18]
 [ 8 31 28 69]]
******************************
Best Epoch: 00380
Training accuracy: 1.3454952239990234
Test_UA: 0.5036255656108597
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  0  3]
 [ 7 42  2  9]
 [14  1 14 21]
 [21 45 13 57]]
******************************
Best Epoch: 00440
Training accuracy: 1.3304256200790405
Test_UA: 0.5279430618401206
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 5  2  3  3]
 [ 1 44  9  6]
 [ 1  2 28 19]
 [ 3 43 31 59]]
******************************
Best Epoch: 00485
Training accuracy: 1.3147125244140625
Test_UA: 0.5397982654600302
Test Confusion Matrix:["ang","sad","hap","neu"]
[[11  2  0  0]
 [11 41  3  5]
 [20  2 12 16]
 [27 48  8 53]]
******************************
Best Epoch: 00445
Training accuracy: 1.3184224367141

In [ ]:
for experiment_num in range(20, 45):
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 20
******************************
Best Epoch: 00390
Training accuracy: 1.3489681482315063
Test_UA: 0.5638838612368025
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 4  2  6  1]
 [ 0 40 19  1]
 [ 0  0 42  8]
 [ 2 24 50 60]]
******************************
Best Epoch: 00340
Training accuracy: 1.2804936170578003
Test_UA: 0.5251602564102564
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [ 9 44  3  4]
 [16  2 15 17]
 [20 49 16 51]]
******************************
Best Epoch: 00220
Training accuracy: 1.4162328243255615
Test_UA: 0.5728846153846154
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 6  6  1  0]
 [ 0 48 10  2]
 [ 5  3 39  3]
 [ 8 60 34 34]]
******************************
Best Epoch: 00455
Training accuracy: 1.3118871450424194
Test_UA: 0.557341628959276
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [ 7 42  8  3]
 [15  0 22 13]
 [18 40 24 54]]
******************************
Best Epoch: 00455
Training accuracy: 1.279763817787

In [ ]:
for experiment_num in range(26, len(params_list_of_dicts)):
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 26
******************************
Best Epoch: 00420
Training accuracy: 1.3050177097320557
Test_UA: 0.5451357466063349
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [ 5 45  6  4]
 [14  5 20 11]
 [20 53 17 46]]
******************************
Best Epoch: 00430
Training accuracy: 1.299932599067688
Test_UA: 0.5113103318250377
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 4  3  6  0]
 [ 0 43 17  0]
 [ 1  1 47  1]
 [ 2 50 73 11]]
******************************
Best Epoch: 00500
Training accuracy: 1.2928990125656128
Test_UA: 0.5286067119155354
Test Confusion Matrix:["ang","sad","hap","neu"]
[[10  2  0  1]
 [10 43  2  5]
 [18  3  9 20]
 [24 41 10 61]]
******************************
Best Epoch: 00495
Training accuracy: 1.3025165796279907
Test_UA: 0.5122285067873303
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  3  2  0]
 [ 6 42 12  0]
 [ 8  3 33  6]
 [22 59 45 10]]
******************************
Best Epoch: 00430
Training accuracy: 1.344047904014

In [ ]:
for experiment_num in range(28, len(params_list_of_dicts)):
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 28
******************************
Best Epoch: 00390
Training accuracy: 1.3334604501724243
Test_UA: 0.5084935897435898
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [13 40  4  3]
 [19  0 15 16]
 [27 45 13 51]]
******************************
Best Epoch: 00125
Training accuracy: 1.7259819507598877
Test_UA: 0.5412820512820513
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  3  2  1]
 [ 2 43 12  3]
 [ 3  0 33 14]
 [ 9 52 41 34]]
******************************
Best Epoch: 00400
Training accuracy: 1.3046125173568726
Test_UA: 0.4883220211161387
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  3  0  1]
 [12 44  1  3]
 [22  2  8 18]
 [31 47  8 50]]
******************************
Best Epoch: 00475
Training accuracy: 1.317643642425537
Test_UA: 0.5468514328808446
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  2  0]
 [ 4 44 12  0]
 [ 9  2 30  9]
 [20 59 35 22]]
******************************
Best Epoch: 00465
Training accuracy: 1.284135460853

In [ ]:
for experiment_num in range(31, len(params_list_of_dicts)):
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
        if accuracy < 0.50:
            break
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 31
******************************
Best Epoch: 00380
Training accuracy: 1.2898304462432861
Test_UA: 0.5106598793363499
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  0  3]
 [ 6 43  4  7]
 [14  1 12 23]
 [19 44  9 64]]
--- Starting trial: 32
******************************
Best Epoch: 00275
Training accuracy: 1.315430998802185
Test_UA: 0.5497134238310709
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  2  3  1]
 [ 3 44 13  0]
 [ 3  3 39  5]
 [ 6 54 56 20]]
--- Starting trial: 33
******************************
Best Epoch: 00305
Training accuracy: 1.2535406351089478
Test_UA: 0.5355128205128206
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  1  2]
 [ 3 43 14  0]
 [10  3 28  9]
 [17 53 32 34]]
--- Starting trial: 34
******************************
Best Epoch: 00220
Training accuracy: 1.5469508171081543
Test_UA: 0.5229487179487179
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  5  1  0]
 [ 1 44 15  0]
 [ 7  2 41  0]
 [12 56 68  0]]
-------------

In [ ]:
for experiment_num in range(31, len(params_list_of_dicts)):
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 500)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
        if accuracy < 0.50:
            break
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 31
******************************
Best Epoch: 00495
Training accuracy: 1.1764452457427979
Test_UA: 0.5100471342383107
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  4  1  0]
 [13 41  5  1]
 [14  5 22  9]
 [27 53 15 41]]
******************************
Best Epoch: 00455
Training accuracy: 1.194471836090088
Test_UA: 0.5362481146304676
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  1  2]
 [ 5 43  9  3]
 [12  1 23 14]
 [18 51 19 48]]
******************************
Best Epoch: 00500
Training accuracy: 1.203068494796753
Test_UA: 0.5265082956259427
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  3  0  1]
 [10 43  4  3]
 [17  1 15 17]
 [21 48 13 54]]
******************************
Best Epoch: 00345
Training accuracy: 1.273468255996704
Test_UA: 0.5501395173453997
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 6  2  3  2]
 [ 2 43 12  3]
 [ 3  0 32 15]
 [ 5 47 32 52]]
******************************
Best Epoch: 00425
Training accuracy: 1.34649550914764

# Experiment

In [ ]:
l1 = [14, 26, 41, 40, 46]
l2 = [4, 20]

In [ ]:
params_list_of_dicts[14]

{'BIAS_INIT': 'ones',
 'BILSTM_UNITS_SPEC': 256,
 'BILSTM_UNITS_TD': 64,
 'BN_CONV1': True,
 'BN_LINEAR': False,
 'BN_MOMENTUM': 0.9,
 'CL_APLHA': 0.5,
 'CL_WEIGHT': 0.01,
 'DIL_RATE_CONV_UFLB': (2, 2),
 'DROP1_RATE': 0.0,
 'DROP2_RATE': 0.0,
 'DROP3_RATE': 0.0,
 'FILTER_CONV1': 128,
 'FILTER_CONV_UFLB': 128,
 'KERNEL_CONV1': (3, 3),
 'KERNEL_CONV_UFLB': (3, 3),
 'KERNEL_INIT': 'glorot_normal',
 'LEAKY_ALPHA': 0.01,
 'LEAKY_CONV_ALT': False,
 'LEAKY_LINEAR': False,
 'LINEAR_UNITS': 0,
 'LR': 1e-05,
 'NUM_DIL_LAYERS': 4,
 'PADDING_CONV1': 'SAME',
 'PADDING_CONV_UFLB': 'SAME',
 'PADDING_POOL_CONV1': 'VALID',
 'POOL_CONV1': 'max',
 'SIZE_POOL_CONV1': (2, 4),
 'STRIDE_CONV1': (1, 1),
 'STRIDE_CONV_UFLB': (1, 1),
 'UNITS_FCN_CONCAT': [64],
 'UNITS_FCN_SPEC': [512, 64],
 'UNITS_FCN_TD': [64]}

In [ ]:
best_experiment = 0
best_accuracy = 0

In [ ]:
for experiment_num in [14, 26, 41, 40, 46]:
    hparams = params_list_of_dicts[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 1000)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 14
******************************
Best Epoch: 00955
Best Valid Accuracy: 0.701680576975415
Test_UA: 0.4982975113122172
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [16 39  4  1]
 [18  1 16 15]
 [30 45 16 45]]
******************************
Best Epoch: 00820
Best Valid Accuracy: 0.6945011890721997
Test_UA: 0.49702300150829565
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [15 37  3  5]
 [20  0 13 17]
 [25 38 16 57]]
******************************
Best Epoch: 00845
Best Valid Accuracy: 0.6668236706367205
Test_UA: 0.5141553544494721
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 9  2  0  2]
 [ 9 43  6  2]
 [19  2 14 15]
 [26 45 15 50]]
******************************
Best Epoch: 00900
Best Valid Accuracy: 0.6695546577214453
Test_UA: 0.4802036199095023
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  2  0  4]
 [12 39  2  7]
 [14  1  5 30]
 [21 26  3 86]]
******************************
Best Epoch: 00960
Best Valid Accuracy: 0.6

In [ ]:
len(params_list_of_dicts_2)

6

In [ ]:
for experiment_num in [20]:
    hparams = params_list_of_dicts_3[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 1000)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 20
******************************
Best Epoch: 00380
Best Valid Accuracy: 0.6714079255640083
Test_UA: 0.483855580693816
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 7  4  1  1]
 [ 9 43  4  4]
 [17  2 16 15]
 [20 53 14 49]]
******************************
Best Epoch: 00400
Best Valid Accuracy: 0.6806300300670027
Test_UA: 0.516395173453997
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  1  2  2]
 [ 9 40  8  3]
 [14  0 23 13]
 [23 43 26 44]]
******************************
Best Epoch: 00595
Best Valid Accuracy: 0.6884156989930147
Test_UA: 0.5249057315233786
Test Confusion Matrix:["ang","sad","hap","neu"]
[[10  1  0  2]
 [15 38  3  4]
 [18  0 15 17]
 [29 40 13 54]]
******************************
Best Epoch: 00845
Best Valid Accuracy: 0.6733750876594249
Test_UA: 0.5214441930618401
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  0  0  5]
 [ 7 41  2 10]
 [16  1  7 26]
 [23 21  4 88]]
******************************
Best Epoch: 00850
Best Valid Accuracy: 0.647

In [ ]:
for experiment_num in [22]:
    hparams = params_list_of_dicts_3[experiment_num]
    print('--- Starting trial: %s' % experiment_num)
    for run in range(5):
        tf.keras.backend.clear_session()
        model = build_model(hparams)
        accuracy = training(model, 1000)
        if best_accuracy < accuracy:
            best_accuracy = accuracy
            best_experiment = experiment_num
    if((experiment_num+1)%5 == 0):
        print('-------------- RESULTS TILL NOW: ------------')
        print('best_experiment: %d' % best_experiment)
        print('best_accuracy' + str(best_accuracy))

--- Starting trial: 22
******************************
Best Epoch: 00620
Best Valid Accuracy: 0.7159943980767486
Test_UA: 0.5086745852187029
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  0  3]
 [11 40  2  7]
 [14  0 13 23]
 [24 33 12 67]]
******************************
Best Epoch: 00855
Best Valid Accuracy: 0.6790335305400397
Test_UA: 0.5062971342383107
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  2  0  3]
 [10 41  6  3]
 [16  0 15 19]
 [29 37 12 58]]
******************************
Best Epoch: 00950
Best Valid Accuracy: 0.6550241630414904
Test_UA: 0.5008559577677225
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  3  0  2]
 [13 44  1  2]
 [18  0  7 25]
 [25 37  4 70]]
******************************
Best Epoch: 00915
Best Valid Accuracy: 0.674500395277407
Test_UA: 0.45982654600301665
Test Confusion Matrix:["ang","sad","hap","neu"]
[[ 8  1  0  4]
 [14 38  1  7]
 [25  0  6 19]
 [33 32  7 64]]
******************************
Best Epoch: 00925
Best Valid Accuracy: 0.6